In [31]:
#| default_exp download

In [32]:
#| export
import wget
from pathlib import Path
from yt_dlp import YoutubeDL
import string as stringlib
from pydub import AudioSegment
import os

In [33]:
url = "https://cdn.changelog.com/uploads/practicalai/236/practical-ai-236.mp3"
episode_name = "practical-ai_236_tech-stack"
episode_type = "podcast"
data_path = os.path.abspath('') + '/../data'

In [34]:
#| export
def unique_dir(dir):
    count = 1
    file, extension = dir.with_suffix(""), dir.suffix
    while dir.exists(): 
        dir = Path(str(file) + f"({count})" + extension)
        count += 1
    return dir

def get_episode_path(episode_name, episode_type, data_path):
    episode_dir = unique_dir(Path(f"{data_path}/{episode_type}/{episode_name}").with_suffix(''))
    return episode_dir/'tmp'/('audio_og.mp3')

def initialise_dirs(episode_path):
    episode_dir = episode_path.parent
    episode_dir.mkdir(parents=True, exist_ok=True)

In [35]:
#| export
def remove_spaces(string, max_words=5, separator="-"): return separator.join(string.split(" ")[:max_words])
def remove_punctuation(string): 
    return string.translate(
        str.maketrans('', '', stringlib.punctuation)
    )

def get_episode_name(url):
    with YoutubeDL({}) as ydl: info = ydl.extract_info(url, download=False)
    name = remove_spaces(remove_punctuation(info['title']))
    channel = remove_spaces(remove_punctuation(info['uploader']), separator="")
    return f"{channel}_{name}"

In [36]:
#| export
def download_youtube(url, episode_path, episode_name=None):

    if episode_name is None: episode_name = get_episode_name(url)

    options = {
    'format':'bestaudio/best',
    'keepvideo':False,
    'outtmpl':str(episode_path),
    # 'postprocessors': [{
    #     'key': 'FFmpegExtractAudio',
    #     'preferredcodec': 'wav',
    #     'preferredquality': '192'
    # }],
    # 'postprocessor_args': [
    #     '-ar', '16000'
    # ],
    'prefer_ffmpeg': True,
    }

    with YoutubeDL(options) as ydl:
        ydl.download([url])

    return episode_path

In [37]:
#| export 
# here we format the audio, and export it both in wav to be used by the transcriber, and a much more copmressed mp3 for the output.
def format_audio(audio_path, spacer=False):
    audio = AudioSegment.from_file(audio_path)
    audio = audio.set_channels(1)
    if spacer:
        audio_spacer = AudioSegment.silent(duration=1000)
        audio = audio_spacer.append(audio, crossfade=0)
    audio_dir = Path(audio_path).parents[1]
    audio_wav = str(audio_dir/'tmp'/'audio.wav')
    audio_mp3 = str(audio_dir/'audio.mp3')
    audio.export(audio_wav, format='wav')
    audio.export(audio_mp3, format='mp3')
    return audio_wav, audio_mp3

In [38]:
#| export
def download_episode(url, episode_name, source, data_path):
    episode_path = get_episode_path(episode_name, source, data_path)
    initialise_dirs(episode_path)
    if source == "podcast":
        episode_path = wget.download(url, out=str(episode_path))
    elif source == "youtube": 
        episode_path = download_youtube(url, episode_path)
    else: 
        raise("Unspecified Source")
    download_paths = format_audio(str(episode_path))
    return download_paths

In [39]:
audio_wav, audio_mp3 = download_episode(url, episode_name, episode_type, data_path)

In [40]:
#| hide
from nbdev import nbdev_export; nbdev_export()